In [1]:
import torch.nn as nn


class ResNet(nn.Module):
    def __init__(self, num_classes=12):
        super(ResNet, self).__init__()
        self.fc1 = nn.Linear(24, 96)
        self.bn1 = nn.BatchNorm1d(96)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(96, 24)
        self.bn2 = nn.BatchNorm1d(24)
        self.fc3 = nn.Linear(24, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        identity = x
        out = self.fc1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.bn2(out)
        out += identity
        out = self.relu(out)
        out = self.fc3(out)
        out = self.softmax(out)
        return out


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/My\ Drive/'Colab Notebooks'

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('iot23_combined_2.csv')


In [5]:
df['label'].value_counts()

PartOfAHorizontalPortScan    100000
Okiru                        100000
Benign                       100000
DDoS                         100000
Name: label, dtype: int64

In [6]:
print(df.columns.tolist())

['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'label', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']


In [7]:
X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]

In [8]:
X.shape

(400000, 24)

In [9]:
Y = pd.get_dummies(df['label']).values

In [10]:
Y.shape

(400000, 4)

In [11]:
from sklearn.preprocessing import MinMaxScaler

In [12]:
scaler = MinMaxScaler()

In [13]:
scaler.fit(X)

MinMaxScaler()

In [14]:
normalized_x = scaler.transform(X)

In [15]:
scaler.fit(Y)

MinMaxScaler()

In [16]:
normalized_y = scaler.transform(Y)

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, Y_train, Y_test = train_test_split(normalized_x, normalized_y, random_state=10, test_size=0.2)

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(Y_train, dtype=torch.float32)
X_val = torch.tensor(X_test, dtype=torch.float32)
y_val = torch.tensor(Y_test, dtype=torch.float32)

# X_train = X_train.clone().detach()
# y_train = y_train.clone().detach()
# X_val = X_val.clone().detach()
# y_val = y_val.clone().detach()

# Create the ResNet model
model = ResNet(num_classes=4)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 300
batch_size = 32
num_batches = len(X_train) // batch_size

for epoch in range(num_epochs):
    running_loss = 0.0
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        inputs = X_train[start_idx:end_idx]
        labels = y_train[start_idx:end_idx]
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    epoch_loss = running_loss / num_batches
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, epoch_loss))

# Evaluate the model on the validation set
with torch.no_grad():
    model.eval()
    outputs = model(X_val)
    _, predicted = torch.max(outputs.data, 1)
    total = y_val.size(0)
    _, normal_y_val = torch.max(y_val.data,1)
    correct = (predicted == normal_y_val).sum().item()
    accuracy = correct / total
    print('Accuracy on the validation set: {:.2f}%'.format(accuracy * 100))


Epoch [1/300], Loss: 1.1744
Epoch [2/300], Loss: 1.1130
Epoch [3/300], Loss: 1.0832
Epoch [4/300], Loss: 1.0700
Epoch [5/300], Loss: 1.0691
Epoch [6/300], Loss: 1.0692
Epoch [7/300], Loss: 1.0683
Epoch [8/300], Loss: 1.0682
Epoch [9/300], Loss: 1.0682
Epoch [10/300], Loss: 1.0689
Epoch [11/300], Loss: 1.0679
Epoch [12/300], Loss: 1.0681
Epoch [13/300], Loss: 1.0679
Epoch [14/300], Loss: 1.0675
Epoch [15/300], Loss: 1.0675
Epoch [16/300], Loss: 1.0675
Epoch [17/300], Loss: 1.0677
Epoch [18/300], Loss: 1.0674
Epoch [19/300], Loss: 1.0672
Epoch [20/300], Loss: 1.0671
Epoch [21/300], Loss: 1.0672
Epoch [22/300], Loss: 1.0682
Epoch [23/300], Loss: 1.0670
Epoch [24/300], Loss: 1.0671
Epoch [25/300], Loss: 1.0669
Epoch [26/300], Loss: 1.0670
Epoch [27/300], Loss: 1.0671
Epoch [28/300], Loss: 1.0671
Epoch [29/300], Loss: 1.0671
Epoch [30/300], Loss: 1.0669
Epoch [31/300], Loss: 1.0671
Epoch [32/300], Loss: 1.0671
Epoch [33/300], Loss: 1.0669
Epoch [34/300], Loss: 1.0669
Epoch [35/300], Loss: 1

In [20]:
torch.save(model,'resnet.pt')